In [1]:
import sys
import os
import threading
import time
import json
from kafka3 import KafkaProducer, KafkaConsumer
from datetime import datetime as dt
from pymongo import MongoClient
from pyspark.sql import Row
import pandas as pd
import os
# add the folder where util.py lives
sys.path.append(os.path.join(os.getcwd(), 'fit3182-a2'))
from util import kafkaProducer

In [2]:
from kafka3.admin import KafkaAdminClient, NewTopic
import time

def recreate_topics(bootstrap_servers, topics,
                    num_partitions=1, replication_factor=1):
    # Ensure we pass a list of host:port strings
    if isinstance(bootstrap_servers, str):
        bs_list = [bootstrap_servers]
    else:
        bs_list = bootstrap_servers

    admin = KafkaAdminClient(bootstrap_servers=bs_list)
    try:
        admin.delete_topics(topics)
        print(f"[INFO] Deletion of topics {topics} initiated.")
    except Exception as e:
        print(f"[WARN] Could not delete topics: {e}")

    time.sleep(2)

    new_topics = [
        NewTopic(name=t, num_partitions=num_partitions, replication_factor=replication_factor)
        for t in topics
    ]
    try:
        admin.create_topics(new_topics)
        print(f"[INFO] Recreated topics {topics}.")
    except Exception as e:
        print(f"[ERROR] Failed to create topics {topics}: {e}")
        raise
    finally:
        admin.close()


In [ ]:
csv_path_a="data/camera_event_A.csv"
csv_path_b="data/camera_event_B.csv"
csv_path_c="data/camera_event_C.csv"

kafka_server="172.22.32.1:9092"

producer_id_a="A"
producer_id_b="B"
producer_id_c="C"

topic_a="camera_event_a"
topic_b="camera_event_b"
topic_c="camera_event_c"

batch_interval=5

topics = ["camera_event_a", "camera_event_b", "camera_event_c"]

# PURGE & RECREATE before producing
recreate_topics(kafka_server, topics,
                num_partitions=1,
                replication_factor=1)

    
producer_a = kafkaProducer(csv_path_a, kafka_server, producer_id_a, topic_a, batch_interval)
producer_b = kafkaProducer(csv_path_b, kafka_server, producer_id_b, topic_b, batch_interval)
producer_c = kafkaProducer(csv_path_c, kafka_server, producer_id_c, topic_c, batch_interval)

threads = []
for p in (producer_a, producer_b, producer_c):
    t = threading.Thread(target=p.produce_batches, daemon=True)
    t.start()
    threads.append(t)

for t in threads:
    t.join()

[INFO] Deletion of topics ['camera_event_a', 'camera_event_b', 'camera_event_c'] initiated.
[INFO] Recreated topics ['camera_event_a', 'camera_event_b', 'camera_event_c'].
[INFO] Successfully loaded data from data/camera_event_A.csv
print 172.22.32.1:9092
[INFO] Successfully loaded data from data/camera_event_B.csv
print 172.22.32.1:9092
[INFO] Successfully loaded data from data/camera_event_C.csv
print 172.22.32.1:9092
172.22.32.1:9092
172.22.32.1:9092
172.22.32.1:9092
[INFO] Publishing batch #1 (20 records)...
[INFO] Publishing batch #1 (5 records)...
[INFO] Publishing batch #1 (1 records)...
[INFO] Batch #1 sent. Sleeping 5s...
[INFO] Batch #1 sent. Sleeping 5s...
[INFO] Batch #1 sent. Sleeping 5s...
[DATA] Batch #1:
                               event_id  batch_id car_plate  camera_id  \
0  d86e8cdb-c387-4ccb-b35a-346302238824         1   UTT 229          3   

                   timestamp  speed_reading  
0 2024-01-01 08:00:54.958092          130.8  

[DATA] Batch #1:
           

[INFO] Publishing batch #4 (2 records)...
[INFO] Batch #4 sent. Sleeping 5s...
[DATA] Batch #4:
                               event_id  batch_id car_plate  camera_id  \
4  55a60e1c-818e-4e55-bc02-0627765629b2         4     ICE 8          3   
5  daafd000-91fa-4ff7-a9bf-896ee74b35ad         4     WC 89          3   

                   timestamp  speed_reading  
4 2024-01-01 08:01:16.405088          108.0  
5 2024-01-01 08:01:18.835653           94.5  

[INFO] Publishing batch #4 (4 records)...
[INFO] Batch #4 sent. Sleeping 5s...
[DATA] Batch #4:
                                event_id  batch_id car_plate  camera_id  \
7   9f2e4a15-24fb-4879-9129-5800338088fa         4    EOT 00          2   
8   ee787113-eb0d-49d6-9cf9-054569625449         4      HP 1          2   
9   cada35fc-33a8-4bf0-a640-e0d6fc2a73df         4     KZE 5          2   
10  ac84bb22-32be-4993-a655-49874adeecc4         4   CJP 278          2   

                    timestamp  speed_reading  
7  2024-01-01 08:00:28.

[INFO] Publishing batch #7 (2 records)...
[INFO] Batch #7 sent. Sleeping 5s...
[DATA] Batch #7:
                               event_id  batch_id car_plate  camera_id  \
8  cc084bd6-f680-43cf-9d6e-15bc481038b2         7  PKH 8115          3   
9  676fa9f2-4a5b-4d1b-a63e-a59b1f2e5c26         7      PI 9          3   

                   timestamp  speed_reading  
8 2024-01-01 08:08:52.583094          153.8  
9 2024-01-01 08:08:53.620450          134.3  

[INFO] Publishing batch #7 (3 records)...
[INFO] Batch #7 sent. Sleeping 5s...
[DATA] Batch #7:
                                event_id  batch_id car_plate  camera_id  \
17  7dff112a-d3ce-4ebb-841f-ea3e03662ead         7    WB 418          2   
18  1f739fcb-8e46-4c92-a69b-9a18533cb646         7      PI 9          2   
19  756a47f3-7f81-4e17-88fa-b2bbed9aa29e         7  PKH 8115          2   

                    timestamp  speed_reading  
17 2024-01-01 08:08:24.091422          138.8  
18 2024-01-01 08:08:26.478501          140.9  
19 2

[INFO] Publishing batch #10 (2 records)...
[INFO] Batch #10 sent. Sleeping 5s...
[DATA] Batch #10:
                                event_id  batch_id car_plate  camera_id  \
13  2f38e970-5042-4d91-8ef3-f8e4bed9908f        10    CZ 592          3   
14  fa9e3b2a-de4e-48be-b97b-1c01e95e437a        10      AH 8          3   

                    timestamp  speed_reading  
13 2024-01-01 08:00:47.453301          161.4  
14 2024-01-01 08:00:49.247531          165.5  

[INFO] Publishing batch #10 (2 records)...
[INFO] Batch #10 sent. Sleeping 5s...
[DATA] Batch #10:
                                event_id  batch_id car_plate  camera_id  \
23  58588fd9-1fd6-4b52-929b-406731bbce2b        10  WPV 1456          2   
24  274dffed-ed69-4f56-8a2e-5c017d27adc1        10     QJ 53          2   

                    timestamp  speed_reading  
23 2024-01-01 08:08:45.845657           75.1  
24 2024-01-01 08:08:46.302895           82.8  

[INFO] Publishing batch #10 (20 records)...
[INFO] Batch #10 sent.

[INFO] Publishing batch #13 (1 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                event_id  batch_id car_plate  camera_id  \
17  49500aa0-39b9-4dc2-8e4d-69c8519b5983        13     JEJ 6          3   

                    timestamp  speed_reading  
17 2024-01-01 08:09:06.369311          109.3  

[INFO] Publishing batch #13 (1 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                event_id  batch_id car_plate  camera_id  \
27  7dba46aa-77b5-47f5-9801-d30e5ddff0d7        13     FO 32          2   

                    timestamp  speed_reading  
27 2024-01-01 08:00:39.240776           94.8  

[INFO] Publishing batch #13 (20 records)...
[INFO] Batch #13 sent. Sleeping 5s...
[DATA] Batch #13:
                                 event_id  batch_id car_plate  camera_id  \
240  2e8b5c14-1d48-484c-8b3e-9500ee86717e        13    HM 258          1   
241  2b480838-5811-4479-b618-23f9d639eb22        13   WG 9

...

[INFO] Publishing batch #127 (1 records)...
[INFO] Batch #127 sent. Sleeping 5s...
[DATA] Batch #127:
                                 event_id  batch_id car_plate  camera_id  \
164  170586b4-6f46-4e3e-a3ad-30617f314e5e       127     NGE 1          3   

                     timestamp  speed_reading  
164 2024-01-01 08:53:08.190588           73.2  

[INFO] Publishing batch #127 (1 records)...
[INFO] Batch #127 sent. Sleeping 5s...
[DATA] Batch #127:
                                 event_id  batch_id car_plate  camera_id  \
244  55be4475-846a-4dd5-b7c3-9093771a3761       127    FR 559          2   

                     timestamp  speed_reading  
244 2024-01-01 09:25:11.922097          150.0  

[INFO] Publishing batch #127 (20 records)...
[INFO] Batch #127 sent. Sleeping 5s...
[DATA] Batch #127:
                                  event_id  batch_id car_plate  camera_id  \
2520  292e6ff7-fcd5-43f9-8c9d-7831b4b71aed       127      AJ 4          1   
2521  80254c34-101b-4c8b-849d-67574306e

[INFO] Publishing batch #130 (1 records)...
[INFO] Batch #130 sent. Sleeping 5s...
[DATA] Batch #130:
                                 event_id  batch_id car_plate  camera_id  \
169  b25f1972-c523-4a95-8f37-569222e244bb       130    WQ 611          3   

                     timestamp  speed_reading  
169 2024-01-01 08:59:05.339482          132.3  

[INFO] Publishing batch #130 (4 records)...
[INFO] Batch #130 sent. Sleeping 5s...
[DATA] Batch #130:
                                 event_id  batch_id car_plate  camera_id  \
249  b1d3c940-9fc4-4c87-91ba-81e1d850e855       130     YHJ 2          2   
250  a81d92c7-b272-474c-b292-12f5411f34f4       130   HJB 322          2   
251  4f729d52-53eb-4d2c-88c5-555c27fd945c       130   DYW 961          2   
252  052f691e-ca4f-4299-97da-23c2db530d2a       130      PS 5          2   

                     timestamp  speed_reading  
249 2024-01-01 09:31:43.466313          139.9  
250 2024-01-01 09:31:43.982350          139.6  
251 2024-01-01 09:31:

[INFO] Publishing batch #133 (2 records)...
[INFO] Batch #133 sent. Sleeping 5s...
[DATA] Batch #133:
                                 event_id  batch_id car_plate  camera_id  \
173  ce6bad13-9ba5-4fd4-909c-dd12e7daca6b       133   THE 567          3   
174  d3b46028-e47d-49cb-b40b-ade620700470       133     HJQ 8          3   

                     timestamp  speed_reading  
173 2024-01-01 09:08:09.852909          115.5  
174 2024-01-01 09:08:11.313944          118.2  

[INFO] Publishing batch #133 (4 records)...
[INFO] Batch #133 sent. Sleeping 5s...
[DATA] Batch #133:
                                 event_id  batch_id car_plate  camera_id  \
257  43b32212-f002-435a-9bde-d24a0ec9ec93       133   MO 7374          2   
258  f2f5dade-e083-49b5-b3e9-19f19a82af02       133    TJZ 83          2   
259  a7303dc2-90b2-496b-aa8f-2d12e491aab4       133      HH 1          2   
260  33c49477-0eb2-46fb-9af9-c81eeffb8bb5       133    VOO 74          2   

                     timestamp  speed_rea

[INFO] Publishing batch #136 (1 records)...
[INFO] Batch #136 sent. Sleeping 5s...
[DATA] Batch #136:
                                 event_id  batch_id car_plate  camera_id  \
177  5054b414-3cf2-4649-8c13-f29287894f5a       136   EHQ 563          3   

                     timestamp  speed_reading  
177 2024-01-01 09:15:46.816108          119.3  

[INFO] Publishing batch #136 (1 records)...
[INFO] Batch #136 sent. Sleeping 5s...
[DATA] Batch #136:
                                 event_id  batch_id car_plate  camera_id  \
263  4762c947-c131-4d89-a3d7-38c3ae672c5b       136     GEI 9          2   

                     timestamp  speed_reading  
263 2024-01-01 09:31:40.458302          142.7  

[INFO] Publishing batch #136 (20 records)...
[INFO] Batch #136 sent. Sleeping 5s...
[DATA] Batch #136:
                                  event_id  batch_id car_plate  camera_id  \
2700  f5bdd7df-f354-46f1-a1a0-c83502bd885b       136    ZFP 37          1   
2701  fba499a9-78ee-4533-8d55-d6d80f7bb

[INFO] Publishing batch #139 (1 records)...
[INFO] Batch #139 sent. Sleeping 5s...
[DATA] Batch #139:
                                 event_id  batch_id car_plate  camera_id  \
181  428e420c-e5bb-4b96-996e-fc09ccf5c480       139      BS 4          3   

                     timestamp  speed_reading  
181 2024-01-01 08:59:02.574849          131.3  

[INFO] Publishing batch #139 (2 records)...
[INFO] Batch #139 sent. Sleeping 5s...
[DATA] Batch #139:
                                 event_id  batch_id car_plate  camera_id  \
266  39b5c809-dbd8-4940-9655-38d89231cdeb       139      ID 3          2   
267  28cea336-77ea-44b7-b933-bd7b139f0e80       139      RS 6          2   

                     timestamp  speed_reading  
266 2024-01-01 09:38:56.927896          110.8  
267 2024-01-01 09:39:00.863625          107.3  

[INFO] Publishing batch #139 (20 records)...
[INFO] Batch #139 sent. Sleeping 5s...
[DATA] Batch #139:
                                  event_id  batch_id car_plate  camer